In [ ]:
# Import libraries:
import pandas as pd
import importlib
import dt_ids7_export_utils as bh_utils
import elfys_functions as bh_elfys

importlib.reload(bh_utils)
importlib.reload(bh_elfys)

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'RRH ELfys Ids7.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH Elfys reduced.xlsx')

del path


# Run all cleanups, filters and checks:
df_ids7 = bh_utils.run_all_cleanup_filters_and_checks(df_ids7, df_dt, True)
# Check for accession numbers that exists in DoseTrack but not in Ids7:
df_dt = bh_utils.check_accession_dt_vs_ids7(df_dt, df_ids7, True)

# First we perform mapping on selected procedures:
df_ids7 = bh_elfys.map_procedures(df_ids7, True)
# Merge the dataframes:
data = bh_utils.merge_ids7_dt(df_ids7, df_dt, True)
# Then we filter the interesting procedures:
data = bh_elfys.filter_procedures(data, True)